# Goal

1. User Input Validation
2. Convert User Input into Customer Query Schema
3. If tools applies, then get the tool response
4. User Query, prompt and tool response, we will perform a next action
5. Support ticket

In [37]:
# Import packages

from pydantic import BaseModel, Field, EmailStr, field_validator
from pydantic_ai import Agent
from typing import Literal, List, Optional # List[str]
from datetime import datetime, date
import json
from openai import OpenAI
import anthropic
import instructor

from dotenv import load_dotenv
load_dotenv(".env", override=True)

import nest_asyncio

nest_asyncio.apply()

In [11]:
import re # regular Expression # regex

pattern = r"[A-Z]{3}-\d{5}" # ABC-29408 DC

print(re.findall(pattern, "123 ZDC-02345 2890 I m teaching ML ZDA-02345") )

print(re.search(pattern, "123 ZDC-02345 2890 I m teaching ML") )

print(re.match(pattern, "ZDC-02345 2890 I m teaching ML") )

['ZDC-02345', 'ZDA-02345']
<re.Match object; span=(4, 13), match='ZDC-02345'>
<re.Match object; span=(0, 9), match='ZDC-02345'>


In [12]:
class UI:
    def __init__(self, name, email):
        self.my_name = name

In [13]:
# Define your UserInput model
class UserInput(BaseModel):
    name: str = Field(..., description="User's name")
    email: EmailStr = Field(..., description="User's email address")
    query: str = Field(..., description="User's query")
    order_id: Optional[str] = Field(None, description="Order ID if available (format: ABC-12345)")
    
    # Validate order_id format (e.g., ABC-12345)
    @field_validator("order_id") # decorator
    def validate_order_id(cls, order_id):
        import re
        if order_id is None:
            return order_id
        pattern = r"^[A-Z]{3}-\d{5}$"
        if not re.match(pattern, order_id):
            raise ValueError(
                "order_id must be in format ABC-12345 "
                "(3 uppercase letters, dash, 5 digits)"
            )
        return order_id
    purchase_date: Optional[date] = None

In [14]:
# Define a function to validate user input
def validate_user_input(user_json: str):
    """Validate user input from a JSON string and return a UserInput 
    instance if valid, otherwise None."""
    try:
        user_input = UserInput.model_validate_json(user_json)
        print("user input validated...")
        return user_input
    except Exception as e:
        print(f" Unexpected error: {e}")
        return None

In [17]:
user_input_json = '''
{
    "name": "Waqas",
    "email": "Waqas@gmail.com",
    "query": "When can I expect delivery of the headphones I ordered?",
    "order_id": "ABC-12345",
    "purchase_date": "2025-12-01"
}
'''
# Validate user input and create a CustomerQuery
valid_data = validate_user_input(user_input_json).model_dump_json()

user input validated...


In [18]:
valid_data

'{"name":"Waqas","email":"Waqas@gmail.com","query":"When can I expect delivery of the headphones I ordered?","order_id":"ABC-12345","purchase_date":"2025-12-01"}'

In [19]:
# Define your CustomerQuery model
class CustomerQuery(UserInput):
    priority: str = Field(
        ..., description="Priority level: low, medium, high"
    )
    category: Literal[
        'refund_request', 'information_request', 'other'
    ] = Field(..., description="Query category")
    is_complaint: bool = Field(
        ..., description="Whether this is a complaint"
    )
    tags: List[str] = Field(..., description="Relevant keyword tags")

In [20]:
# Define a function to call an LLM using Pydantic AI to create an instance of CustomerQuery
def create_customer_query(valid_user_json: str) -> CustomerQuery:
    
    customer_query_agent = Agent(
        model="google-gla:gemini-2.0-flash",
        output_type=CustomerQuery,
    )
    response = customer_query_agent.run_sync(valid_user_json)
    print("CustomerQuery generated...")
    return response.output

In [22]:
# Define user input JSON data

customer_query = create_customer_query(valid_data)
print(type(customer_query))
print()
print(customer_query.model_dump_json(indent=2))

CustomerQuery generated...
<class '__main__.CustomerQuery'>

{
  "name": "Waqas",
  "email": "Waqas@gmail.com",
  "query": "When can I expect delivery of the headphones I ordered?",
  "order_id": "ABC-12345",
  "purchase_date": "2025-12-01",
  "priority": "medium",
  "category": "information_request",
  "is_complaint": false,
  "tags": [
    "delivery",
    "headphones"
  ]
}


In [27]:
# Create a fake FAQ database as a list of entries with keywords
faq_db = [
    {
        "question": "How can I reset my password?",
        "answer": "To reset your password, click 'Forgot Password' on the sign-in page and follow the instructions sent to your email.",
        "keywords": ["password", "reset", "account"]
    },
    {
        "question": "How long does shipping take?",
        "answer": "Standard shipping takes 3-5 business days. You can track your order in your account dashboard.",
        "keywords": ["shipping", "delivery", "order", "tracking"]
    },
    {
        "question": "How can I return an item?",
        "answer": "You can return any item within 30 days of purchase. Visit our returns page to start the process.",
        "keywords": ["return", "refund", "exchange"]
    },
    {
        "question": "How can I delete my account?",
        "answer": "To delete your account, go to your account settings tab and select 'delete account'.",
        "keywords": ["delete", "account", "remove"]
    }
]

# Create a fake order database
order_db = {
    
    "ABC-12345": {
        "status": "shipped", "estimated_delivery": "2025-12-05",
        "purchase_date": "2025-12-01", "email": "waqas@gmail.com"
    },
    "XYZ-23456": {
        "status": "processing", "estimated_delivery": "2025-12-15",
        "purchase_date": "2025-12-10", "email": "salman@example.com"
    },
    "QWE-34567": {
        "status": "delivered", "estimated_delivery": "2025-12-20",
        "purchase_date": "2025-12-18", "email": "ali@example.com"
    }
    
}



In [28]:
# Define FAQ Lookup tool input as a Pydantic model
class FAQLookupArgs(BaseModel):
    query: str = Field(..., description="User's query") 
    tags: List[str] = Field(
        ..., description="Relevant keyword tags from the customer query"
    )

In [29]:
# Define your FAQ lookup tool

def lookup_faq_answer(args: FAQLookupArgs) -> str: # query:str, tags: ["laptop", "password"]
    """Look up an FAQ answer by matching tags and words in query 
    to FAQ entry keywords."""
    query_words = set(word.lower() for word in args.query.split())
    tag_set = set(tag.lower() for tag in args.tags)
    
    best_match = None # dictionary
    best_score = 0
    for faq in faq_db: # faq = dict
        keywords = set(k.lower() for k in faq["keywords"])
        score = len(keywords & tag_set) + len(keywords & query_words)
        if score > best_score:
            best_score = score
            best_match = faq
    if best_match and best_score > 0:
        return best_match["answer"]
    return "Sorry, I couldn't find an FAQ answer for your question."

In [24]:
# Define Check Order Status tool input as a Pydantic model
class CheckOrderStatusArgs(BaseModel): # pydantic base model
    order_id: str = Field(
        ..., description="Customer's order ID (format: ABC-12345)"
    )
    email: EmailStr = Field(..., description="Customer's email address")

    @field_validator("order_id")
    def validate_order_id(cls, order_id):
        import re
        pattern = r"^[A-Z]{3}-\d{5}$"
        if not re.match(pattern, order_id):
            raise ValueError(
                "order_id must be in format ABC-12345 "
                "(3 uppercase letters, dash, 5 digits)"
            )
        return order_id

In [26]:
# Define your check order status tool
def check_order_status(args: CheckOrderStatusArgs) -> dict: # order_id:str, email: w@g.c
    """Simulate checking the status of a customer's order by 
    order_id and email."""
    order = order_db.get(args.order_id) #  return dict  {"status": "shipped", "estimated_delivery": "2025-12-05""purchase_date": "2025-12-01", "email": "waqas@gmail.com"

    if not order:
        return {
            "order_id": args.order_id,
            "status": "not found",
            "estimated_delivery": None,
            "note": "order_id not found"
        }
        
    elif args.email.lower() != order.get("email", "").lower():
        return {
            "order_id": args.order_id,
            "status": order["status"],
            "estimated_delivery": order["estimated_delivery"],
            "note": "order_id found but email mismatch"
        }
        
    return {
        "order_id": args.order_id,
        "status": order["status"],
        "estimated_delivery": order["estimated_delivery"],
        "note": "order_id and email match"
    } 

In [30]:
FAQLookupArgs.model_json_schema()

{'properties': {'query': {'description': "User's query",
   'title': 'Query',
   'type': 'string'},
  'tags': {'description': 'Relevant keyword tags from the customer query',
   'items': {'type': 'string'},
   'title': 'Tags',
   'type': 'array'}},
 'required': ['query', 'tags'],
 'title': 'FAQLookupArgs',
 'type': 'object'}

In [31]:
# Define tools for your API call

# tool schema 

tool_definitions = [
    {
        "type": "function",
        "function": {
            "name": "lookup_faq_answer",
            "description": "Look up an FAQ answer by matching tags to FAQ entry keywords.",
            "parameters": FAQLookupArgs.model_json_schema() # query:str, tags: list[str]
        }
    },
    {
        "type": "function",
        "function": {
            "name": "check_order_status",
            "description": "Check the status of a customer's order.",
            "parameters": CheckOrderStatusArgs.model_json_schema()
        }
    }
]

In [33]:
tool_definitions

[{'type': 'function',
  'function': {'name': 'lookup_faq_answer',
   'description': 'Look up an FAQ answer by matching tags to FAQ entry keywords.',
   'parameters': {'properties': {'query': {'description': "User's query",
      'title': 'Query',
      'type': 'string'},
     'tags': {'description': 'Relevant keyword tags from the customer query',
      'items': {'type': 'string'},
      'title': 'Tags',
      'type': 'array'}},
    'required': ['query', 'tags'],
    'title': 'FAQLookupArgs',
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'check_order_status',
   'description': "Check the status of a customer's order.",
   'parameters': {'properties': {'order_id': {'description': "Customer's order ID (format: ABC-12345)",
      'title': 'Order Id',
      'type': 'string'},
     'email': {'description': "Customer's email address",
      'format': 'email',
      'title': 'Email',
      'type': 'string'}},
    'required': ['order_id', 'email'],
    'title': 'CheckOr

### Define your support ticket output model

In [34]:
#Define your final output Pydantic models
class OrderDetails(BaseModel):
    status: str
    estimated_delivery: str
    note: str

class SupportTicket(CustomerQuery):
    recommended_next_action: Literal[
        'escalate_to_agent', 'send_faq_response', 
        'send_order_status', 'no_action_needed'
    ] = Field(
        ..., description="LLM's recommended next action for support"
    )
    order_details: Optional[OrderDetails] = Field(
        None, description="Order details if action is send_order_status"
    )
    faq_response: Optional[str] = Field(
        None, description="FAQ response if action is send_faq_response"
    )
    creation_date: datetime = Field(
        ..., description="Date and time the ticket was created"
    )

In [39]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [42]:
print(json.dumps(SupportTicket.model_json_schema(), indent=2))

{
  "$defs": {
    "OrderDetails": {
      "properties": {
        "status": {
          "title": "Status",
          "type": "string"
        },
        "estimated_delivery": {
          "title": "Estimated Delivery",
          "type": "string"
        },
        "note": {
          "title": "Note",
          "type": "string"
        }
      },
      "required": [
        "status",
        "estimated_delivery",
        "note"
      ],
      "title": "OrderDetails",
      "type": "object"
    }
  },
  "properties": {
    "name": {
      "description": "User's name",
      "title": "Name",
      "type": "string"
    },
    "email": {
      "description": "User's email address",
      "format": "email",
      "title": "Email",
      "type": "string"
    },
    "query": {
      "description": "User's query",
      "title": "Query",
      "type": "string"
    },
    "order_id": {
      "anyOf": [
        {
          "type": "string"
        },
        {
          "type": "null"
        }
 

In [44]:
customer_query.model_dump()

{'name': 'Waqas',
 'email': 'Waqas@gmail.com',
 'query': 'When can I expect delivery of the headphones I ordered?',
 'order_id': 'ABC-12345',
 'purchase_date': datetime.date(2025, 12, 1),
 'priority': 'medium',
 'category': 'information_request',
 'is_complaint': False,
 'tags': ['delivery', 'headphones']}

In [61]:
getattr(customer_query, "emai", None)

In [62]:
# Initialize OpenAI client
client = OpenAI()

# Define a function to call OpenAI with tools
def decide_next_action_with_tools(customer_query: CustomerQuery):
    
    support_ticket_schema = json.dumps(SupportTicket.model_json_schema(), indent=2)
    
    system_prompt = f"""
        You are a helpful customer support agent. Your job is to 
        determine what support action should be taken for the customer, 
        based on the customer query and the expected fields in the 
        SupportTicket schema below. If more information on a particular 
        order_id or FAQ response would be helpful in responding to the 
        user query and can be obtained by calling a tool, call the 
        appropriate tool to get that information. If an order_id is 
        present in the query, always look up the order status to get 
        more information on the order.

        Here is the JSON schema for the SupportTicket model you must 
        use as context for what information is expected:
        {support_ticket_schema}
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": str(customer_query.model_dump())}
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tool_definitions,
        tool_choice="auto"
    )
    message = response.choices[0].message
    tool_calls = getattr(message, "tool_calls", None)
    return message, tool_calls, messages

In [56]:
customer_query(valid_data)

TypeError: 'CustomerQuery' object is not callable

In [63]:
tool_calls

[ChatCompletionMessageFunctionToolCall(id='call_Szb9bH53W8yLtgJB5WSvOVjh', function=Function(arguments='{"order_id":"ABC-12345","email":"Waqas@gmail.com"}', name='check_order_status'), type='function')]

### Inspect the LLM's outputs and tool calls

In [66]:
# Call the decide_next_action_with_tools function
message, tool_calls, messages = decide_next_action_with_tools(customer_query)
"Investigate the LLM's outputs before proceeding"

print("LLM message:\n", json.dumps(message.model_dump(), indent=2))
print(
    "\nTool calls:\n", 
    json.dumps([call.model_dump() for call in tool_calls], indent=2)
)

LLM message:
 {
  "content": null,
  "refusal": null,
  "role": "assistant",
  "annotations": [],
  "audio": null,
  "function_call": null,
  "tool_calls": [
    {
      "id": "call_0e4IbdhhiIyb4QinxTfb0Kom",
      "function": {
        "arguments": "{\"order_id\":\"ABC-12345\",\"email\":\"Waqas@gmail.com\"}",
        "name": "check_order_status"
      },
      "type": "function"
    }
  ]
}

Tool calls:
 [
  {
    "id": "call_0e4IbdhhiIyb4QinxTfb0Kom",
    "function": {
      "arguments": "{\"order_id\":\"ABC-12345\",\"email\":\"Waqas@gmail.com\"}",
      "name": "check_order_status"
    },
    "type": "function"
  }
]


In [49]:
print(messages)

[{'role': 'system', 'content': '\n        You are a helpful customer support agent. Your job is to \n        determine what support action should be taken for the customer, \n        based on the customer query and the expected fields in the \n        SupportTicket schema below. If more information on a particular \n        order_id or FAQ response would be helpful in responding to the \n        user query and can be obtained by calling a tool, call the \n        appropriate tool to get that information. If an order_id is \n        present in the query, always look up the order status to get \n        more information on the order.\n\n        Here is the JSON schema for the SupportTicket model you must \n        use as context for what information is expected:\n        {\n  "$defs": {\n    "OrderDetails": {\n      "properties": {\n        "status": {\n          "title": "Status",\n          "type": "string"\n        },\n        "estimated_delivery": {\n          "title": "Estimated Del

In [52]:
print(type(message))

message.model_dump()

<class 'openai.types.chat.chat_completion_message.ChatCompletionMessage'>


{'content': None,
 'refusal': None,
 'role': 'assistant',
 'annotations': [],
 'audio': None,
 'function_call': None,
 'tool_calls': [{'id': 'call_Szb9bH53W8yLtgJB5WSvOVjh',
   'function': {'arguments': '{"order_id":"ABC-12345","email":"Waqas@gmail.com"}',
    'name': 'check_order_status'},
   'type': 'function'}]}

In [77]:
tool_calls[0].function.name

'check_order_status'

In [72]:
tool_calls[0].function.arguments

'{"order_id":"ABC-12345","email":"Waqas@gmail.com"}'

In [78]:
tool_calls[0].id

'call_0e4IbdhhiIyb4QinxTfb0Kom'

In [75]:
args = CheckOrderStatusArgs.model_validate_json(tool_calls[0].function.arguments)

check_order_status(args)

{'order_id': 'ABC-12345',
 'status': 'shipped',
 'estimated_delivery': '2025-12-05',
 'note': 'order_id and email match'}

### Gather tool outputs and prepare for ticket generation

In [79]:

# Define a function to get tool outputs

def get_tool_outputs(tool_calls): # list
    tool_outputs = []
    if tool_calls:
        for tool_call in tool_calls:
            if tool_call.function.name == "lookup_faq_answer":
                print("Agent requested a call to the Lookup FAQ tool...")
                args = FAQLookupArgs.model_validate_json(
                    tool_call.function.arguments
                )
                result = lookup_faq_answer(args)
                tool_outputs.append({
                    "tool_call_id": tool_call.id, "output": result
                })
                print(f"Lookup FAQ tool returned {result}")
                
            elif tool_call.function.name == "check_order_status":
                print("Agent requested a call to Check Order Status tool...")
                args = CheckOrderStatusArgs.model_validate_json(
                    tool_call.function.arguments
                )
                result = check_order_status(args)
                tool_outputs.append({
                    "tool_call_id": tool_call.id, "output": result
                })
                print(f"Check Order Status tool returned {result}")
    return tool_outputs

tool_outputs = get_tool_outputs(tool_calls)

# Print tool outputs for inspection
print("Tool outputs:\n", json.dumps(tool_outputs, indent=2))

Agent requested a call to Check Order Status tool...
Check Order Status tool returned {'order_id': 'ABC-12345', 'status': 'shipped', 'estimated_delivery': '2025-12-05', 'note': 'order_id and email match'}
Tool outputs:
 [
  {
    "tool_call_id": "call_0e4IbdhhiIyb4QinxTfb0Kom",
    "output": {
      "order_id": "ABC-12345",
      "status": "shipped",
      "estimated_delivery": "2025-12-05",
      "note": "order_id and email match"
    }
  }
]


In [84]:
"\n".join([f"Tool: {out['tool_call_id']} Output: {json.dumps(out['output'])}" for out in tool_outputs]) if tool_outputs else "No tool calls were made."

'Tool: call_rpuosewf8cFfCacCjZeVImg0 Output: {"order_id": "QWE-34567", "status": "delivered", "estimated_delivery": "2025-12-20", "note": "order_id found but email mismatch"}'

In [14]:
# "\n".join([f"Tool: {out['tool_call_id']} Output: {json.dumps(out['output'])}" for out in tool_outputs]) if tool_outputs else "No tool calls were made."

### Generate a structured support ticket using Anthropic

In [80]:
# Create the Anthropic client with Instructor
# anthropic_client = instructor.from_anthropic(
#     anthropic.Anthropic()
# )

# Define a function to call Anthropic to generate a support ticket
def generate_structured_support_ticket(customer_query: CustomerQuery, 
                                       message, tool_outputs: list):
    
    tool_results_str = "\n".join([f"Tool: {out['tool_call_id']} Output: {json.dumps(out['output'])}" for out in tool_outputs]) if tool_outputs else "No tool calls were made."
    # Concatenate prompt parts into a single string for Anthropic
    prompt = f"""
        You are a support agent. Use all information below to 
        generate a support ticket as a validated Pydantic model.
        Customer query: {customer_query.model_dump_json(indent=2)}
        LLM message: {str(message.content)}
        Tool results: {tool_results_str}
    """
    # Create the message with structured output
    response = client.responses.parse(
    model="gpt-4o",
    max_output_tokens=1024,
    input=[{"role": "user", "content": prompt}],
    text_format=SupportTicket)
    
    # response = anthropic_client.messages.create(
    #     model="claude-3-7-sonnet-latest",  
    #     max_tokens=1024,
    #     messages=[
    #         {
    #             "role": "user", 
    #             "content": prompt
    #         }
    #     ],
    #     response_model=SupportTicket
    # )
    
    support_ticket = response
    support_ticket.creation_date = datetime.now()
    return support_ticket

### Print your final support ticket

In [81]:
# Run the final step of generating a support ticket and print output
support_ticket = generate_structured_support_ticket(customer_query, message,
                                                    tool_outputs)
print(support_ticket.model_dump_json(indent=2))

{
  "id": "resp_0749c1de2fed16ce0068fdd48844a081a19f145c74e947815a",
  "created_at": 1761465480.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4o-2024-08-06",
  "object": "response",
  "output": [
    {
      "id": "msg_0749c1de2fed16ce0068fdd489278c81a1ac10a0c3c763f9a1",
      "content": [
        {
          "annotations": [],
          "text": "{\"name\":\"Waqas\",\"email\":\"Waqas@gmail.com\",\"query\":\"When can I expect delivery of the headphones I ordered?\",\"order_id\":\"ABC-12345\",\"purchase_date\":\"2025-12-01\",\"priority\":\"medium\",\"category\":\"information_request\",\"is_complaint\":false,\"tags\":[\"delivery\",\"headphones\"],\"recommended_next_action\":\"send_order_status\",\"order_details\":{\"status\":\"shipped\",\"estimated_delivery\":\"2025-12-05\",\"note\":\"order_id and email match\"},\"faq_response\":null,\"creation_date\":\"2023-11-02T12:00:00Z\"}",
          "type": "output_text",
          "logpr

### Full workflow: validate, query, decide, tool, and generate ticket

In [82]:
# Define new user input data
user_json = '''
{
    "name": "Waqas",
    "email": "waqas@gmail.com",
    "query": "I'm really not happy with this product I bought",
    "order_id": "QWE-34567",
    "purchase_date": null
}
'''

In [83]:
# Run the entire pipeline
valid_user_json = validate_user_input(user_json).model_dump_json()

customer_query = create_customer_query(valid_user_json)

message, tool_calls, messages = decide_next_action_with_tools(customer_query)

tool_outputs = get_tool_outputs(tool_calls)

support_ticket = generate_structured_support_ticket(customer_query, message, tool_outputs)

print(support_ticket.model_dump_json(indent=2))

user input validated...
CustomerQuery generated...
Agent requested a call to Check Order Status tool...
Check Order Status tool returned {'order_id': 'QWE-34567', 'status': 'delivered', 'estimated_delivery': '2025-12-20', 'note': 'order_id found but email mismatch'}
{
  "id": "resp_020bec1b98dd93ed0068fdd4c79560819fbdd981757d91b117",
  "created_at": 1761465544.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4o-2024-08-06",
  "object": "response",
  "output": [
    {
      "id": "msg_020bec1b98dd93ed0068fdd4c985cc819f990e4beb6b04627e",
      "content": [
        {
          "annotations": [],
          "text": "{\"name\":\"Waqas\",\"email\":\"waqas@gmail.com\",\"query\":\"I'm really not happy with this product I bought\",\"order_id\":\"QWE-34567\",\"purchase_date\":null,\"priority\":\"high\",\"category\":\"other\",\"is_complaint\":true,\"tags\":[\"product issue\"],\"recommended_next_action\":\"escalate_to_agent\",\"order_detail

In [ ]:
1. Customer Query - >  User Input -> validate -> CusotomerQuery-> CusotomerQuery Instance
2. LLM -> Customer Query -> message -> tools -> content/tool call
3. tool_call -> id, name, args
4. main -> 2 functions -> function output
5. Customer Query, message, function output
6 Support Ticket